# Lab 7: Object-Oriented Python

## Overview

After last time's lab, which covered the basics of OOP, we'll continue playing with classes today, writing a fair chunk of code and building several classes to solve a variety of problems.

Recall these definitions to guide you through this lab:

- An *object* has identity
- A *name* is a reference to an object
- A *namespace* is an associative mapping from names to objects
- An *attribute* is any name following a dot ('.')

## Part #1 Concrete Object Oriented Programming

Enough monkeying around! Let's have you work on a number of objects that you may one day encounter in your programmer life.

### Exercise #1 Not-so-SimpleGraph

In this part, you'll build the implementation for a `SimpleGraph` class in Python.

You will need to define a `Vertex` class, an `Edge` class, and a `SimpleGraph` class. The specification is as follows:

(1) A `Vertex` has attributes:

* `name`, a string representing the label of the vertex.
* `edges`, a set representing edges outbound from this vertex to its neighbors

A new Vertex should be initialized with an optional `name`, which defaults to `""`, and should be initialized with an empty edge set.
Also with methods such as `__str__`, `__repr__` to make the print more readable. **You are free to add any other methods that considered useful.**

(2) An `Edge` has attributes:

* `start`, a `Vertex` representing the start point of the edge.
* `end`, a `Vertex` representing the end point of the edge.
* `cost`, a `float` (used for graph algorithms) representing the weight of the edge.
* `visited`, a `bool` (used for graph algorithms) representing whether this edge has been visited before.

Note that for our purposes, an `Edge` is directed: the edge from `v_a` to `v_b` is distinct from the edge from `v_b` to `v_a`. **You can make the direction explict by using arrows such as `-->` in the `__str__` method.**

An `Edge` requires a `start` and `end` vertex in order to be instantiated. `cost` should default to 1, and `visited` should default to `False`, but both should be able to be set via an initializer.

(3) A `SimpleGraph` has attributes

* `verts`, a collection of `Vertex`s (you need to decide the collection type)
* `edges`, a collection of `Edge`s (you need to decide the collection type)

as well as several methods:

* `graph.add_vertex(v)`
* `graph.add_edge(v_1, v_2)`
* `graph.contains_vertex(v)` - return `True` or `False`
* `graph.contains_edge(v_1, v_2)` - return `True` or `False`
* `graph.get_neighbors(v)` - return get a list of incoming neighbors and a list of outgoing neighbors
* `graph.is_empty()` - return `True` or `False`
* `graph.size()` - return number of edges |E|
* `graph.remove_vertex(v)` **tip: first remove the edges attached to the vertex**
* `graph.remove_edge(v_1, v_2)`
* `graph.is_neighbor(v1, v2)` - return `True` or `False`; direction of neighborhood does not matter here
* `graph.is_reachable(v1, v2)` - return `True` or `False`, use any algorithm you like
* `graph.clear_all()`

The actual implementation details are up to you.

*Note: debugging will be significantly easier if you write `__str__` or `__repr__` methods on your custom classes.*

In [4]:
class Vertex:
    def __init__(self, name="", edges=None):
        self.name = name
        self.edges = set() if edges is None else edges

    def __str__(self):
        edges_str = sorted(self.edges, key=lambda edge: edge.cost)
        return f"V({self.name}) ---{', '.join(str(edge.cost) for edge in edges_str)}---> [{', '.join(str(edge.end.name) for edge in edges_str)}]"

    def __repr__(self):
        return f"V({self.name})"


class Edge:
    def __init__(self, start: Vertex, end: Vertex, cost=1, visited=False):
        self.start = start
        self.end = end
        self.cost = cost
        self.visited = visited

    def __eq__(self, other):
        return (
            isinstance(other, Edge)
            and self.start == other.start
            and self.end == other.end
            and self.cost == other.cost
        )

    def __hash__(self):
        return hash((self.start, self.end, self.cost))

    def __str__(self):
        return f"{self.start.name} ---{self.cost}---> {self.end.name}"

    def __repr__(self):
        return f"{self.start.name}---{self.cost}-->{self.end.name}"


class SimpleGraph:
    def __init__(self, verts=None, edges=None):
        if edges is None:
            edges = set()
        if verts is None:
            verts = set()
        self.verts = verts
        self.edges = edges

    def __str__(self):
        string = "---- SimpleGraph ----{\n"
        for edge in self.edges:
            string += f"V({edge.start.name}) ---{edge.cost}---> V({edge.end.name})\n"
        return string

    def add_vertex(self, v):
        self.verts.add(v)

    def add_edge(self, v_1, v_2, cost=1):
        edge = (v_1, v_2, cost)
        if edge not in self.edges:
            self.edges.add(edge)
            v_1.edges.add(edge)
            v_2.edges.add(edge)

    def contains_vertex(self, v):
        return v in self.verts

    def contains_edge(self, v_1, v_2):
        return any(
            (edge.start == v_1 and edge.end == v_2) or (edge.start == v_2 and edge.end == v_1)
            for edge in self.edges
        )

    def get_neighbors(self, vertex):
        in_neighbors = [edge.start for edge in self.edges if edge.end == vertex]
        out_neighbors = [edge.end for edge in self.edges if edge.start == vertex]
        return in_neighbors, out_neighbors

    def is_empty(self):
        return not bool(self.verts)

    def size(self):
        return len(self.edges)

    def remove_vertex(self, vertex):
        if vertex in self.verts:
            edges_to_remove = [edge for edge in self.edges if edge.start == vertex or edge.end == vertex]
            for edge in edges_to_remove:
                self.edges.remove(edge)
                edge.start.edges.remove(edge)
                edge.end.edges.remove(edge)
            self.verts.remove(vertex)

    def remove_edge(self, v_1, v_2):
        edge_to_remove = next(
            (edge for edge in self.edges if
             (edge.start == v_1 and edge.end == v_2) or (edge.start == v_2 and edge.end == v_1)),
            None,
        )
        if edge_to_remove:
            self.edges.remove(edge_to_remove)
            v_1.edges.remove(edge_to_remove)
            v_2.edges.remove(edge_to_remove)

    def is_neighbor(self, v1, v2):
        return any(
            (edge.start == v1 and edge.end == v2) or (edge.start == v2 and edge.end == v1)
            for edge in self.edges
        )

    def is_reachable(self, v1, v2):
        visited = set()
        stack = [v1]
        while stack:
            top = stack.pop()
            if top == v2:
                return True
            if top not in visited:
                visited.add(top)
                stack.extend(edge.end for edge in top.edges if edge.start == top and edge.end not in visited)
        return False

    def clear_all(self):
        self.verts.clear()
        self.edges.clear()

In [5]:
rand_graph = [
    ('1', '2', 0.7),
    ('1', '3', 0.2),
    ('1', '4', 0.3),
    ('2', '3', 0.3),
    ('2', '5', 0.4),
    ('3', '5', 0.4),
    ('3', '8', 0.5),
    ('4', '12', 0.2),
    ('5', '6', 0.1),
    ('6', '8', 0.3),
    ('7', '8', 0.2),
    ('7', '13', 0.2),
    ('9', '10', 0.6),
    ('9', '11', 0.4),
    ('9', '12', 0.4),
    ('10', '12', 0.1),
    ('11', '13', 0.5),
]
verts = {}
edges = set()
for v1, v2, cost in rand_graph:
    start = Vertex(v1)
    end = Vertex(v2)
    edge = Edge(start, end, cost)
    # print(edge)
    # print(start)
    # print(end)
    edges.add(edge)
    if v1 not in verts:
        verts[v1] = start
    if v2 not in verts:
        verts[v2] = end
    if edge not in verts[v1].edges:
        verts[v1].edges.add(edge)

# construct SimpleGraph with set of verts and edges
graph = SimpleGraph(set(verts.values()), edges)
print(graph.verts)
print(graph.edges)
print(graph)

print(graph.is_empty())
# False
print(graph.size())
# 17
print(graph.verts)
# {V(6), V(11), V(12), V(13), V(9), V(2), V(3), V(1), V(10), V(7), V(4), V(5), V(8)}

print("===== Reachability =====")
print(graph.is_reachable(verts["1"], verts["7"])) # False
print(graph.is_reachable(verts["1"], verts["2"])) # True
print(graph.is_reachable(verts["1"], verts["8"])) # True
print("===== Are neighbors? =====")
print(graph.is_neighbor(verts["7"], verts["8"])) #True
print(graph.is_neighbor(verts["8"], verts["7"])) #True
print(graph.is_neighbor(verts["6"], verts["12"])) #False

{V(1), V(7), V(2), V(6), V(5), V(10), V(12), V(11), V(3), V(9), V(4), V(13), V(8)}
{1---0.2-->3, 7---0.2-->8, 9---0.6-->10, 9---0.4-->12, 11---0.5-->13, 2---0.3-->3, 7---0.2-->13, 10---0.1-->12, 1---0.3-->4, 3---0.4-->5, 4---0.2-->12, 9---0.4-->11, 5---0.1-->6, 3---0.5-->8, 6---0.3-->8, 1---0.7-->2, 2---0.4-->5}
---- SimpleGraph ----{
V(1) ---0.2---> V(3)
V(7) ---0.2---> V(8)
V(9) ---0.6---> V(10)
V(9) ---0.4---> V(12)
V(11) ---0.5---> V(13)
V(2) ---0.3---> V(3)
V(7) ---0.2---> V(13)
V(10) ---0.1---> V(12)
V(1) ---0.3---> V(4)
V(3) ---0.4---> V(5)
V(4) ---0.2---> V(12)
V(9) ---0.4---> V(11)
V(5) ---0.1---> V(6)
V(3) ---0.5---> V(8)
V(6) ---0.3---> V(8)
V(1) ---0.7---> V(2)
V(2) ---0.4---> V(5)

False
17
{V(1), V(7), V(2), V(6), V(5), V(10), V(12), V(11), V(3), V(9), V(4), V(13), V(8)}
===== Reachability =====
False
True
False
===== Are neighbors? =====
False
False
False


#### Some example to test your clases on

In [6]:
# Testing code
# graph-like data for testing
rand_graph = [
    ('1', '2', 0.7),
    ('1', '3', 0.2),
    ('1', '4', 0.3),
    ('2', '3', 0.3),
    ('2', '5', 0.4),
    ('3', '5', 0.4),
    ('3', '8', 0.5),
    ('4', '12', 0.2),
    ('5', '6', 0.1),
    ('6', '8', 0.3),
    ('7', '8', 0.2),
    ('7', '13', 0.2),
    ('9', '10', 0.6),
    ('9', '11', 0.4),
    ('9', '12', 0.4),
    ('10', '12', 0.1),
    ('11', '13', 0.5),
]

# construct set of verts and edges with above data
verts = {}
edges = set()
for v1, v2, cost in rand_graph:
    start = Vertex(v1)
    end = Vertex(v2)
    edge = Edge(start, end, cost)
    edges.add(edge)
    if v1 not in verts:
        verts[v1] = start
    if v2 not in verts:
        verts[v2] = end
    if edge not in verts[v1].edges:
        verts[v1].edges.add(edge)

# construct SimpleGraph with set of verts and edges
graph = SimpleGraph(set(verts.values()), edges)

# Some tests
print("===== Basic structure ====")
print(graph)
# Results could be a visualization such as:
# ---- SimpleGraph ----{
# V(1) ---0.3---> V(4),
# V(6) ---0.3---> V(8),
# V(7) ---0.2---> V(13),
# V(10) ---0.1---> V(12),
# V(2) ---0.4---> V(5),
# V(11) ---0.5---> V(13),
# V(2) ---0.3---> V(3),
# V(5) ---0.1---> V(6),
# V(9) ---0.4---> V(12),
# V(1) ---0.2---> V(3),
# V(7) ---0.2---> V(8),
# V(9) ---0.4---> V(11),
# V(3) ---0.4---> V(5),
# V(3) ---0.5---> V(8),
# V(1) ---0.7---> V(2),
# V(9) ---0.6---> V(10),
# V(4) ---0.2---> V(12)}
print(graph.is_empty())
# False
print(graph.size())
# 17
print(graph.verts)
# {V(6), V(11), V(12), V(13), V(9), V(2), V(3), V(1), V(10), V(7), V(4), V(5), V(8)}

print("===== Reachability =====")
print(graph.is_reachable(verts["1"], verts["7"])) # False
print(graph.is_reachable(verts["1"], verts["2"])) # True
print(graph.is_reachable(verts["1"], verts["8"])) # True
print("===== Are neighbors? =====")
print(graph.is_neighbor(verts["7"], verts["8"])) #True
print(graph.is_neighbor(verts["8"], verts["7"])) #True
print(graph.is_neighbor(verts["6"], verts["12"])) #False
print("===== All neighbors =====")
print(graph.get_neighbors(verts["1"])) # ([], [V(3), V(2), V(4)])
print(graph.get_neighbors(verts["3"])) # ([V(2), V(1)], [V(5), V(8)])
print("===== Remove edge =====")
graph.remove_edge(Vertex("1"), Vertex("3"))
print(graph.size())
# 16
print(graph.get_neighbors(verts["1"]))
# ([], [V(4), V(2)])
print("===== Remove vertex =====")
graph.remove_vertex(Vertex("2"))
print(graph.size()) #13
print(graph)
# ---- SimpleGraph ----{
# V(1) ---0.3---> V(4),
# V(6) ---0.3---> V(8),
# V(7) ---0.2---> V(13),
# V(10) ---0.1---> V(12),
# V(11) ---0.5---> V(13),
# V(5) ---0.1---> V(6),
# V(9) ---0.4---> V(12),
# V(7) ---0.2---> V(8),
# V(9) ---0.4---> V(11),
# V(3) ---0.4---> V(5),
# V(3) ---0.5---> V(8),
# V(9) ---0.6---> V(10),
# V(4) ---0.2---> V(12)}
# ---------------------

===== Basic structure ====
---- SimpleGraph ----{
V(2) ---0.3---> V(3)
V(1) ---0.2---> V(3)
V(9) ---0.6---> V(10)
V(6) ---0.3---> V(8)
V(9) ---0.4---> V(12)
V(9) ---0.4---> V(11)
V(5) ---0.1---> V(6)
V(2) ---0.4---> V(5)
V(4) ---0.2---> V(12)
V(10) ---0.1---> V(12)
V(11) ---0.5---> V(13)
V(3) ---0.5---> V(8)
V(1) ---0.7---> V(2)
V(7) ---0.2---> V(13)
V(3) ---0.4---> V(5)
V(7) ---0.2---> V(8)
V(1) ---0.3---> V(4)

False
17
{V(10), V(1), V(6), V(9), V(2), V(7), V(12), V(11), V(3), V(5), V(4), V(13), V(8)}
===== Reachability =====
False
True
False
===== Are neighbors? =====
False
False
False
===== All neighbors =====
([], [V(2)])
([V(1)], [])
===== Remove edge =====
17
([], [V(2)])
===== Remove vertex =====
17
---- SimpleGraph ----{
V(2) ---0.3---> V(3)
V(1) ---0.2---> V(3)
V(9) ---0.6---> V(10)
V(6) ---0.3---> V(8)
V(9) ---0.4---> V(12)
V(9) ---0.4---> V(11)
V(5) ---0.1---> V(6)
V(2) ---0.4---> V(5)
V(4) ---0.2---> V(12)
V(10) ---0.1---> V(12)
V(11) ---0.5---> V(13)
V(3) ---0.5---> V(8)


### Exercise #2: Implementing Graph Algorithms

If you're feeling up to the challenge, and you have sufficient time, goimplement other graph algorithms, using your SimpleGraph from exercise #6. The point isn't to check whether you know your graph algorithms - rather, these algorithms will serve to test the correctness of your graph implementation. The particulars are up to you.

As some suggestions:

* [Longest path](https://en.wikipedia.org/wiki/Longest_path_problem)
* [Dijkstra's Algorithm](https://en.wikipedia.org/wiki/Dijkstra%27s_algorithm)
* [A*](https://en.wikipedia.org/wiki/A*_search_algorithm)
* [Max Flow](https://en.wikipedia.org/wiki/Maximum_flow_problem)
* <a href="https://en.wikipedia.org/wiki/Clique_(graph_theory)">K-Clique</a>
* <a href="https://en.wikipedia.org/wiki/Component_(graph_theory)">Largest Connected Component</a>
* [is_bipartite](https://en.wikipedia.org/wiki/Bipartite_graph#Testing_bipartiteness)
* [hamiltonian_path_exists](https://en.wikipedia.org/wiki/Hamiltonian_path)


In [ ]:
# Here is a sample run for Dijkstra's algorithm
print(graph.dijsktra('1', '8')) # V(1) to V(8) has a shortest path [V(1), V(3), V(8)]
print(graph.dijsktra('1', '13')) # V(1) to V(13) path not feasible !

### Exercise #3: My Magical Graph

See if you can rewrite the `SimpleGraph` class from exercise #6, using magic methods to emulate the behavior and operators of standard Python. In particular,

```
graph[v]  # returns neighbors of v
graph[v] = v_2  # Insert an edge from v to v2
len(graph) # size of the graph
# etc
```

If you've tackled exercise #6, make sure to modify your code there as well!


### Exercise #4: Timed Key-Value Store

Let's build an interesting data structure straight out of an interview programming challenge from [Stripe](https://stripe.com/). This is more of an algorithms challenge than a Python challenge, but we hope you're still interested in tackling it.

At a high-level, we'll be building a key-value store (think `dict` or Java's `HashMap`) that has a `get` method that takes an optional second parameter as a `time` object in Python to return the most recent value before that period in time. If no key-value pair was added to the map before that period in time, return `None`. We'll call this class `TimedKVStore`.

You’ll need some sort of `time` object to track when key-value pairs are getting added to this map. Consider using [the `time` module](https://docs.python.org/3/library/time.html).

To give you an idea of how this class works, this is what should happen after you implement `TimedKVStore`.

```Python
d = TimedKVStore()
t0 = time.time()
d.put("1", 1)
t1 = time.time()
d.put("1", 1.1)
d.get("1")  # => 1.1
d.get("1", t1)  # => 1
d.get("1", t0)  # => None
```

In [ ]:
class TimedKVStore:
    pass

d = TimedKVStore()
t0 = time.time()
d.put("1", 1)
t1 = time.time()
d.put("1", 1.1)
print(d.get("1"))  # => 1.1
print(d.get("1", t1))  # => 1
print(d.get("1", t0))  # => None

### Exercise #5: Remove

Implement a method on a `TimedKVStore` to `remove(key)` that takes a key and removes that entire key from the key-value store.

Write another `remove(key, time)` method that takes a key and removes all memory of values before that time method.

## Part #2 Magic Methods

### Reading

Python provides an enormous number of special "magic" methods that a class can override to interoperator with builtin Python operations. You can skim through an [approximate visual list](http://diveintopython3.problemsolving.io/special-method-names.html) from Dive into Python3, or a [more verbose explanation](https://rszalski.github.io/magicmethods/), or the [complete Python documentation](https://docs.python.org/3/reference/datamodel.html#specialnames) on special methods.

Fair warning, there are a lot of them, so it's probably better to skim than to really take a deep dive, unless you're loving this stuff. It's still a good thing to have a general idea of what exists and what can be done.

### Exercise #6: Polynomial Class

We will write a `Polynomial` class that acts like a number. As a a reminder, a [polynomial](https://en.wikipedia.org/wiki/Polynomial) is a mathematical object that looks like $1 + x + x^2$ or $4 - 10x + x^3$ or $-4 - 2x^{10}$. A mathematical polynomial can be evaluated at a given value of $x$. For example, if $f(x) = 1 + x + x^2$, then $f(5) = 1 + 5 + 5^2 = 1 + 5 + 25 = 31$.

Polynomials are also added componentwise: If $f(x) = 1 + 4x + 4x^3$ and $g(x) = 2 + 3x^2 + 5x^3$, then $(f + g)(x) = (1 + 2) + 4x + 3x^2 + (4 + 5)x^3 = 3 + 4 + 3x^2 + 9x^3$.

Construct a polynomial with variadic coefficients keywords: the zeroth coefficient is given with `c_0`, the square coefficient is given with `c_2`, and so on. For example, `f = Polynomial(c_0=1, c_2=3, c_4=5)` should construct a `Polynomial` representing $1 + 3x^2 + 5x^4$.

You will need to override the addition special method (`__add__`) and the callable special method (`__call__`).

You should be able to emulate the following code:

```Python
f = Polynomial(c_0=1, c_1=5, c_2=10)
g = Polynomial(c_0=1, c_1=3, c_2=5)

print(f(5))  # => Invokes `f.__call__(5)`
print(g(2))  # => Invokes `g.__call__(2)`

h = f + g    # => Invokes `f.__add__(g)`
print(h(3))  # => Invokes `h.__call__(3)`
```

Lastly, implement a method to convert a `Polynomial` to an informal string representation. For example, the polynomial `Polynomial(1, 3, 5)` should be represented by the string `"1 * x^0 + 3 * x^1 + 5 * x^2"`.

**Bonus** (you'll get bonus marks if you implement this correctly): Implement also the special method `__sub__` to subtract two polynomials, and `__mul__` which lets you multiply a polynomial either by a scalar or another polynomial (see sample code below for examples)

In [5]:
# A class to represent polynomials of any degree

class Polynomial:
    def __init__(self, **coeff):
        # **coeff - initialization with variable number of coefficients
        # store the coeff in a dictionary with the power as the key
        self.coefficients = {int(key.split('_')[-1]): value for key, value in coeff.items()}

    def __call__(self, x):
        """Estimate the polynomial with a given value of x"""
        # calculate each term and sum them
        return sum(coeff * (x ** power) for power, coeff in self.coefficients.items())

    def __add__(self, other):
        """Sum of 2 polynomials"""
        # create a new polynomial with the same coeff
        result = Polynomial(**{f"c_{power}": coeff for power, coeff in self.coefficients.items()})
        # iterate over the coeff of the other
        for power, coeff in other.coefficients.items():
            # add the coeff of the same power or create a new entry if it doesn't exist
            result.coefficients[power] = result.coefficients.get(power, 0) + coeff
        return result

    def __sub__(self, other):
        """Subtract two polynomials"""
        # create a new polynomial with the same coeff
        result = Polynomial(**{f"c_{power}": coeff for power, coeff in self.coefficients.items()})
        # iterate over the coeff of the other
        for power, coeff in other.coefficients.items():
            # subtract the coef of the same power or create a new entry if it doesn't exist
            result.coefficients[power] = result.coefficients.get(power, 0) - coeff
        return result

    def __mul__(self, other):
        """Multiply two polynomials or a polynomial and a scalar"""
        # create an empty polynomial
        result = Polynomial()
        # check the type of other: polinomial or scalar
        if isinstance(other, Polynomial):
            # iterate over the coeff of both polynomials
            for power_1, coeff_1 in self.coefficients.items():
                for power_2, coeff_2 in other.coefficients.items():
                    power = power_1 + power_2
                    # ddd the product term to the result or create a new entry if it doesn't exist
                    result.coefficients[power] = result.coefficients.get(power, 0) + coeff_1 * coeff_2
        else:
            for power, coeff in self.coefficients.items():
                # multiply the coefficient by the scalar and store it
                result.coefficients[power] = coeff * other
        return result

    def __str__(self):
        terms = [f"{coeff} * x^{power}" if power > 0 else f"{coeff}" for power, coeff in self.coefficients.items()]
        return " + ".join(terms)

In [6]:
# Testing code
f = Polynomial(c_0=1, c_1=5, c_2=10)
g = Polynomial(c_0=1, c_1=3, c_2=5)
h = Polynomial(c_0=1, c_5=7)

print(f"given f = {f}; f(5) = {f(5)}")
print(f"given g = {g}; g(2) = {g(2)}")
print(f"given h = {h}; g(3) = {g(3)}")
# given f = 1 + 5x + 10x^2; f(5) = 276
# given g = 1 + 3x + 5x^2; g(2) = 27
# given h = 1 + 7x^5; g(3) = 55

print(f"({f}) + ({g}) = {f + g}")
print(f"({f}) + ({h}) = {f + h}")

# Optional: subtraction and multiplication
print(f"({f}) - ({g}) = {f - g}")
print(f"({f})5 = {f * 5}")
print(f"({f})({g}) = {f * g}")
# (1 + 5x + 10x^2) - (1 + 3x + 5x^2) = 2x + 5x^2
# (1 + 5x + 10x^2)5 = 5 + 25x + 50x^2
# (1 + 5x + 10x^2)(1 + 3x + 5x^2) = 1 + 8x + 30x^2 + 55x^3 + 50x^4

given f = 1 + 5 * x^1 + 10 * x^2; f(5) = 276
given g = 1 + 3 * x^1 + 5 * x^2; g(2) = 27
given h = 1 + 7 * x^5; g(3) = 55
(1 + 5 * x^1 + 10 * x^2) + (1 + 3 * x^1 + 5 * x^2) = 2 + 8 * x^1 + 15 * x^2
(1 + 5 * x^1 + 10 * x^2) + (1 + 7 * x^5) = 2 + 5 * x^1 + 10 * x^2 + 7 * x^5
(1 + 5 * x^1 + 10 * x^2) - (1 + 3 * x^1 + 5 * x^2) = 0 + 2 * x^1 + 5 * x^2
(1 + 5 * x^1 + 10 * x^2)5 = 5 + 25 * x^1 + 50 * x^2
(1 + 5 * x^1 + 10 * x^2)(1 + 3 * x^1 + 5 * x^2) = 1 + 8 * x^1 + 30 * x^2 + 55 * x^3 + 50 * x^4


### Exercise #7: Polynomial Extensions

If you are looking for more, implement even more additional operations on our `Polynomial` class from exercise #6. You may want to implement `__div__`.

You can also implement more complicated mathematical operations, such as `f.derivative()`, which returns a new function that is the derivative of `f`, or `.zeros()`, which returns a collection of the function's zeros.

If you need even more, write a `classmethod` to construct a polynomial from a string representation of it. You should be able to write:

```
f = Polynomial.parse("1 * x^0 + 3 * x^1 + 5 * x^2")
```

### Exercise #8: `MultivariatePolynomial`

Write a class called `MultivariatePolynomial` that represents a polynomial in many variables. For example, $f(x, y, z) = 4xy + 10x^2z - 5x^3yz + y^4z^3$ is a polynomial in three variables.

How would you provide coefficients to the constructor? How would you define the arguments to the callable? How would you implement the mathematical operations efficiently?

You should keep in mind the `Polynomial` class from exercise #6.

In [ ]:
# your implementation here
class MultivariatePolynomial:
    pass

## Part #3: Review: Functions as objects

As you know from class, everything in Python is an object. Including functions. Functions have some interesting attributes to explore. We'll poke around several of these attributes more in depth here. You already saw a couple of attributes in lab5, such as `__doc__`.

Usually, this information isn't particularly useful for practitioners (you'll rarely want to hack around with the internals of functions), but even seeing that you *can* in Python is very cool. And it *will* be useful on the event you *really* need to monkeypatch a given functionality.

In this section, there is no code to write. Instead, you will be reading and running code and observing the output. Nevertheless, we encourage you to play around with the code cells to experiment and explore on your own.

#### Default Values (`__defaults__` and `__kwdefaults__`)

As stated earlier, any default values (either normal default arguments or the keyword-only default arguments that follow a variadic positional argument parameter) are bound to the function object at the time of function definition. Consider our `all_together` function from earlier, and run the following code. Why might the `__defaults__` attribute be a tuple, but the `__kwdefaults__` attribute be a dictionary?

In [ ]:
def all_together(x, y, z=1, *nums, indent=True, spaces=4, **options):
    pass

all_together.__defaults__  # => (1, )
all_together.__kwdefaults__  # => {'indent':True, 'spaces':4}

#### Code Object (`__code__`)

In CPython, the reference implementation of Python used by many people (including us), functions are byte-compiled into executable Python code, or _bytecode_, when defined. This code object, which represents the bytecode and some administrative information, is bound to the `__code__` attribute, and has a ton of interesting properties, best illustrated by example. Code objects are immutable and contain no references to immutable objects.

```Python
def all_together(x, y, z=1, *nums, indent=True, spaces=4, **options):
    """A useless comment"""
    print(x + y * z)
    print(sum(nums))
    for k, v in options.items():
        if indent:
            print("{}\t{}".format(k, v))
        else:
            print("{}{}{}".format(k, " " * spaces, v))
            
code = all_together.__code__
```

| Attribute  | Sample Value | Explanation |
| --- | --- | --- |
| `code.co_argcount` | `3` | number of positional arguments (including arguments with default values) |
| `code.co_cellvars` | `()` | tuple containing the names of local variables that are referenced by nested functions |
| `code.co_code` | `b't\x00\x00...\x04S\x00'` | string representing the sequence of bytecode instructions |
| `code.co_consts` | `('A useless comment', '{}\t{}', '{}{}{}', ' ', None)` | tuple containing the literals used by the bytecode - our `None` is from the implicit `return None` at the end |
| `code.co_filename` | `filename` or `<stdin>` or `<ipython-input-#-xxx>` | file in which the function was defined |
| `code.co_firstlineno` | `1` | line of the file the first line of the function appears |
| `code.co_flags` | `79` | AND of compiler-specific binary flags whose internal meaning is (mostly) opaque to us |
| `code.co_freevars` | `()` | tuple containing the names of free variables |
| `code.co_kwonlyargcount` | `2` | number of keyword-only arguments |
| `code.co_lnotab` | `b'\x00\x02\x10\x01\x0c\x01\x12\x01\x04\x01\x12\x02'` | string encoding the mapping from bytecode offsets to line numbers |
| `code.co_name` | `"all_together"` | the function name  |
| `code.co_names` | `('print', 'sum', 'items', 'format')` | tuple containing the names used by the bytecode |
| `code.co_nlocals` | `9` | number of local variables used by the function (including arguments) |
| `code.co_stacksize` | `7` | required stack size (including local variables) |
| `code.co_varnames` | `('x', 'y', 'z', 'indent', 'spaces', 'nums', 'options', 'k', 'v')` | tuple containing the names of the local variables (starting with the argument names) |

More info on this, and on all types in Python, can be found at the [data model reference](https://docs.python.org/3/reference/datamodel.html#the-standard-type-hierarchy). For code objects, you have to scroll down to "Internal Types."

In [ ]:
def all_together(x, y, z=1, *nums, indent=True, spaces=4, **options):
    """A useless comment"""
    print(x + y * z)
    print(sum(nums))
    for k, v in options.items():
        if indent:
            print("{}\t{}".format(k, v))
        else:
            print("{}{}{}".format(k, " " * spaces, v))

code = all_together.__code__

print(code.co_argcount)
print(code.co_cellvars)
print(code.co_code)
print(code.co_consts)
print(code.co_filename)
print(code.co_firstlineno)
print(code.co_flags)
print(code.co_freevars)
print(code.co_kwonlyargcount)
print(code.co_lnotab)
print(code.co_name)
print(code.co_names)
print(code.co_nlocals)
print(code.co_stacksize)
print(code.co_varnames)

##### Security

This can lead to a pretty glaring security vulnerability. Namely, the code object on a given function can be hot-swapped for the code object of another (perhaps malicious function) at runtime!

In [ ]:
def nice(): print("You're awesome!")
def mean(): print("You're... not awesome. OOOOH")

# Overwrite the code object for nice
nice.__code__ = mean.__code__

print(nice())  # prints "You're... not awesome. OOOOH"

##### `dis` module

The `dis` module, for "disassemble," exports a `dis` function that allows us to disassemble Python byte code (at least, for Python distributions implemented in CPython for existing versions). The disassembled code isn't exactly normal assembly code, but rather is a specialized Python syntax

```Python
def gcd(a, b):
    while b:
        a, b = b, a % b
    return a
    
import dis
dis.dis(gcd)
"""
  2           0 SETUP_LOOP              27 (to 30)
        >>    3 LOAD_FAST                1 (b)
              6 POP_JUMP_IF_FALSE       29

  3           9 LOAD_FAST                1 (b)
             12 LOAD_FAST                0 (a)
             15 LOAD_FAST                1 (b)
             18 BINARY_MODULO
             19 ROT_TWO
             20 STORE_FAST               0 (a)
             23 STORE_FAST               1 (b)
             26 JUMP_ABSOLUTE            3
        >>   29 POP_BLOCK

  4     >>   30 LOAD_FAST                0 (a)
             33 RETURN_VALUE
"""
```

Details on the instructions themselves can be found [here](https://docs.python.org/3/library/dis.html#python-bytecode-instructions).
You can read more about the `dis` module [here](https://docs.python.org/3/library/dis.html).

In [ ]:
def gcd(a, b):
    while b:
        a, b = b, a % b
    return a

import dis
dis.dis(gcd)

#### Call (`__call__`)

All Python functions have a `__call__` attribute, which is the actual object called when you use parentheses to "call" a function. That is,

In [ ]:
def greet(): print("Hello world!")

greet() # "Hello world!"
# is just syntactic sugar for
greet.__call__()  # "Hello world!"

This means that any object (including instances of custom classes) with a `__call__` method can use the parenthesized function call syntax. Which is why we wrote the following to construct a callable `Polynomial` class:

```Python
class Polynomial:
    def __init__(self, coeffs):
        """Store the coefficients..."""
        
    def __call__(self, x):
        """Compute f(x)..."""


# The polynomial f(x) = 4 + 4 * x + 4 * x ** 2
f = Polynomial(4, 4, 1)
f(5)  # Really, this is f.__call__(5)
```

#### Name Information (`__module__`, `__name__`, and `__qualname__`)

Python functions also store some name information about a function, generally for the purposes of friendly printing.

`__module__` refers to the module that was active at the time the function was defined. Any functions defined in the interactive interpreter, or run as a script, will have `__module__ == '__main__'`, but imported modules will have their `__module__` attribute set to the module name. For example, `math.sqrt.__module__` is `"math"`.

`__name__` is the function's name. Nothing special here.

`__qualname__`, which stands for "qualified name," only differs from `__name__` when you're dealing with nested functions.


# Submission instructions

You will need to exercises #1 `Not-so-SimpleGraph` and #6 `Polynomial Class` on Arche before **9:59am on Friday, 8th December 2023**. Submit either a `.py` or an `.ipynb` file containing the exercises and name it `td7_firstname_lastname_grpN.py` or `td7_firstname_lastname_grpN.ipynb` accordingly, where `firstname` should be your first name, `lastname` should be your last name, and `N` in `grpN` should be your group number (e.g. Jane Doe, who is in group A1, should name her submission either `td7_jane_doe_grp1.py` or `td7_jane_doe_grp1.ipynb`, depending on whether Jane submitted a Python script or a Jupyter notebook).

To evaluate your submission, we will be looking at the following criteria:

- Does your code run? (So **run** your program at least once before submitting!)
- Does it run correctly? (So **test** your solution with a few different inputs!)
- Is your code well-commented?

> Stolen from CS Stanford. With <3 by @sredmond

> Adapted by tmickus